In [1]:
import torch.nn as nn
import torch 
import torch.nn.functional as F
import math, copy, re
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import torchtext
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")
print(torch.__version__)

2.0.0


**Creating Embedding Vectors**

- Each embedding vector's size is 512. If the vocab size is 100, that makes the embedding matrix 100x512. Similarly, if batch_size=64, the output will be 64x10x512.

In [2]:
class EmbeddingVectors(nn.Module):
    def __init__(self, vocab_size, embedding_dimension):
        """
        Parameters:
            vocab_size: vocabulary size
            embedding_dimension: dimension of embeddings
        """
        super(EmbeddingVectors, self).__init__()
        self.embed = nn.EmbeddingVectors(vocab_size, embedding_dimension)
    
    def fforward(self, x):
        """
        Parameters:
            x: input vector
        Returns:
            output: embedding vector
        """
        output = self.embed(x)
        return output

**Positional Encoding**
- In the paper, authors use sin() and cos() to create positional encoding. According to the paper, cosine is used for odd time steps and sine used in even time steps.
- PE = sin(pos/10000^2i/d)
- PE = cos(pos/10000^2i/d)
- Where pos is order in the sentence and i is position. If we have a batch size of 32 and seq length of 10 and let embedding dimension be 512, positional encoding vector's dimension would be 32x10x512. Then we add embedding vector's dimension with this.

In [3]:
class PositionalEmbedding(nn.Module):
    def __init__(self, max_seq_len, embed_model_dimension):
        super(PositionalEmbedding, self).__init__()
        self.embed_dim = embed_model_dimension
        
        positional_encoding = torch.zeros(max_seq_len, self.embed_dim)
        for pos in range(max_seq_len):
            for i in range(0, self.embed_dim, 2):
                positional_encoding[pos, i] = np.sin(pos / (10000 ** ((2 * i) / self.embed_dim)))
                positional_encoding[pos, i + 1] = np.cos(pos / (10000 ** ((2 * (i)) / self.embed_dim)))
        positional_encoding = positional_encoding.unsqueeze(0)
        self.register_buffer('pe', positional_encoding)
        
    def fforward(self, x):
        x = x * np.sqrt(self.embed_dim)
        seq_len = x.size(1)
        x = x + torch.autograd.Variable(self.positional_encoding[:, :seq_len], requires_grad=False)
        return x

In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embedding_dimension=512, n_heads=8):
        super(MultiHeadAttention, self).__init__()
        
        self.embedding_dimension = embedding_dimension
        self.n_heads = n_heads
        self.single_head_dim = int(self.embedding_dimension / self.n_heads) #512 / 8 = 6 each key, query, value will be of 64d
        
        #key, query, value matrices 64 x 64
        self.query_matrix = nn.Linear(self.single_head_dim, self.single_head_dim, bias=False) 
        self.key_matrix = nn.Linear(self.single_head_dim, self.single_head_dim, bias=False)
        self.value_matrix = nn.Linear(self.n_heads * self.single_head_dim, self.embed_dim)
        self.out = nn.Linear(self.n_heads * self.single_head_dim, self.embed_dim)
        
    def fforward(self, key, query, value, mask=None):
        batch_size = key.size(0)
        seq_length = key.size(1)
        
        seq_length_query = query.size(1)
        
        key = key.view(batch_size, seq_length, self.n_heads, self.single_head_dim)
        query = query.view(batch_size, seq_length_query, self.n_heads, self.single_head_dim)
        value = value.view(batch_size, seq_length, self.n_heads, self.single_head_dim)
        
        key_ = self.key_matrix(key)
        query_ = self.query_matrix(query)
        value_ = self.value_matrix(value)
        
        query_ = query_.transpose(1,2)
        key_ = key.transpose(1,2)
        value_ = key.transpose(1,2)
        
        #computing attention
        key_adjusted = key_.transpose(-1,2)
        product = torch.matmul(query_, key_adjusted) #(32 x 8 x 10 x 64) x (32 x 8 x 64 x 10)
        
        if mask is not None:
            product = product.masked_fill(mask == 0, float("-1e20"))
            
        product = product / math.sqrt(self.single_head_dim) #/sqrt(64)
        
        #softmax
        scores = F.softmax(product, dim=-1)
        
        #multiply with value matrix
        scores = torch.matmul(scores, v)
        
        #concatenate
        concat = scores.transpose(1,2).contiguous().view(batch_size, seq_length_query, self.single_head_dim * self.n_heads)
        
        output = self.out(concat) #(32, 10, 512) -> (32, 10, 512)
        
        return output